In [1]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import time

#chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

# try:
#     driver = webdriver.Chrome(f'91/chromedriver')   
# except:
#     chromedriver_autoinstaller.install(True)
#     driver = webdriver.Chrome(f'91/chromedriver')

driver = webdriver.Chrome(f'../chromedriver')



In [21]:
driver.get("https://v4.map.naver.com")
driver.find_elements_by_css_selector("button.btn_close")[1].click()
# spmc_zoomout
driver.implicitly_wait(10)
#         a:nth-child(3)#

search_box = driver.find_element_by_css_selector("input#search-input")
search_box.send_keys("남대문")

search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)
search_box.clear()

#지도레벨 축소
# for i in range(3):    
#     driver.find_elements_by_css_selector("a.spmc_zoomout")[0].click()
#     time.sleep(0.25)


#print(chrome_ver)

## store ids 
store_ids = [];

In [18]:
store_id_path = 'store_id.txt'

def get_store_id(webl_obj) :
    for store in webl_obj:
        str_id = store.get_attribute("class") + ""
        store_ids.append(str_id.split(',')[1]);
        print(str_id.split(',')[1])
        WriteTxtFiles(store_id_path, str_id.split(',')[1])


def WriteTxtFiles(path, store_id):    
    with open(path, "a") as file:
        file.write(store_id + ",")
        file.close()



In [22]:
search_query  = ['식당']
search_box = driver.find_element_by_css_selector("input#search-input")
search_box.clear()
current_page_num = 1;
max_search_page_num = 50;

search_box.send_keys(search_query)
#         a:nth-child(3)#
search_button = driver.find_element_by_css_selector("button.spm")
search_button.click()
time.sleep(1)

# 우선 store_id는 100개만 수집
for i in range(max_search_page_num):
    next_button = driver.find_element_by_css_selector("div.paginate").find_elements_by_tag_name("a")    
    stores = driver.find_elements_by_css_selector("ul.lst_site>li")
    get_store_id(stores)
    current_page_num += 1;
    
    for nextItem in next_button:
        if(nextItem.text == "다음"):
            nextItem.click()
            time.sleep(1)
            break
            
        if(nextItem.text == str(current_page_num)):
            nextItem.click()
            time.sleep(1)
            break
 

1240466794
1725002504
15100983
20136886
1557591983
15100843
15100782
15100776
15366750
13495681
38767337
19937048
1823619000
1589818280
1473824048
1687801777
1932753741
1339398284
33877589
680542670
1380222194
15367065
1958670920
36649063
36133037
36651678
1848932798
37259814
15102303
1961426380
1168201710
1159093203
32045663
1180720323
12076089
1431765910
1661457261
15105524
15100716
1908332868
15085364
32046322
1591279841
1945404929
1398605717
1773158515
1342858165
1134058731
1061381929
1699874050
15074379
1626314303
15083760
15098573
15100546
682020960
36369949
1114143873
35848791
13028670
1012080500
1095747833
31428153
1345259227
1027853149
1291075256
11831368
1927375586
1695145422
196326357
38532228
15111060
1037631212
37039697
1318552498
447138355
1268166809
15094564
36276341
31825005
20130131
1908746293
20007073
20379819
1887600198
1192177162
1537910454
1785158441
32625059
1053815272
12801773
15094205
1507299388
1036071392
37936738
891500898
1423679422
15075672
13376539
11748805

KeyboardInterrupt: 

In [23]:
# 수집된 store_id로 방문자 리뷰 데이터를 수집한다
from bs4 import BeautifulSoup
import requests

with open("store_id.txt", "r") as file:
    store_ids = file.read()
    file.close()
    
driver = webdriver.Chrome(f'../chromedriver')

for store_id in store_ids.split(","):
    if(store_id):
        print("================")
        
        url = f'https://pcmap.place.naver.com/restaurant/{store_id}/review/visitor'
        htm = driver.get(url)
        time.sleep(1)
        
        try:
            for i in range(20):
                more_btn = driver.find_element_by_class_name("_3iTUo")
                if(more_btn):
                    more_btn.click()
                    time.sleep(0.5)
                else:
                    break;
        except Exception as ex:
            print("error catch : " , ex)
        
        print("id : " + store_id)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # li _2Cv-r -> 부모 컨테이너 // span _2tObC  ->별점 // span WoYOw ->리뷰 내용

        for ll_item in soup.find_all('li',{'class' : '_2Cv-r'}):

            rating_ll = ll_item.find('span',{'class' : '_2tObC'})
            review_ll = ll_item.find('span',{'class' : 'WoYOw'})

            if(review_ll):
                #10자 이상 리뷰만 의미있는 값으로 본다.
                if(len(review_ll.text)>10):
                    #3점 ~ 4점대 리뷰는 거른다 (3점미만 : 부정 , 5점 : 긍정)
                    if(float(rating_ll.text) < 3 or float(rating_ll.text) > 4.5):
                        review_content = review_ll.text
                        label = "0" #0 : 긍정, 1 부정
                        if("\n" in review_ll.text):
                            review_content = review_ll.text.replace("\n"," ")
                            
                        #3점 미만이면 부정평가
                        if(float(rating_ll.text) < 3 ):
                            label = "1";
                            
                            with open("review_data.txt", "a") as file:
                                # id , 별점, 리뷰내용, 레이블(0:긍정, 1:부정)
                                file.write(store_id+"\t" + rating_ll.text+"\t" + review_content +"\t"+ label+ "\n") 
    #                     print("rating : " + rating_ll.text)
    #                     print("review : " + review_ll.text)
        file.close()
#        driver.close()
        

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1240466794
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1725002504
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15100983
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 20136886
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1557591983
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)



error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1773158515
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1342858165
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1134058731
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1061381929
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1699874050
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.10

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 12801773
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15094205
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1507299388
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1036071392
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 37936738
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 12001256
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15084705
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15080700
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 38305587
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 15075484
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1010258673
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 12793501
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1685924836
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1262835367
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1489782740
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 11383977
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 34636148
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 37696135
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 1149997082
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id : 35202049
error catch :  Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3iTUo"}
  (Session info: chrome=91.0.4472.101)

id :